In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3052,application_1732639283265_3010,pyspark,idle,Link,Link,None,
3082,application_1732639283265_3038,pyspark,idle,Link,Link,None,
3084,application_1732639283265_3040,pyspark,idle,Link,Link,None,
3087,application_1732639283265_3043,pyspark,idle,Link,Link,None,
3089,application_1732639283265_3045,pyspark,idle,Link,Link,None,
3100,application_1732639283265_3056,pyspark,idle,Link,Link,None,
3102,application_1732639283265_3058,pyspark,idle,Link,Link,None,
3112,application_1732639283265_3068,pyspark,idle,Link,Link,None,
3116,application_1732639283265_3072,pyspark,idle,Link,Link,None,
3125,application_1732639283265_3081,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession,Row
# for the execution time
import time

results = []

spark = SparkSession.builder.appName("Query 1 RDD").getOrCreate()
conf = spark.sparkContext.getConf()

# Print  executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

spark.catalog.clearCache()
# Read both files
csv_files = [
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
]
crime_data_df = spark.read.csv(csv_files, header=True, inferSchema=True)
start_time = time.time()
crime_data_rdd = crime_data_df.rdd
aggregated_data = crime_data_rdd.map(lambda x: [int(x[11]),x[9]]) \             #keep only the columns that we need
    .filter(lambda x: x[0] > 0 and  x[1] and 'AGGRAVATED ASSAULT' in x[1]) \    #filter based on criteria
    .map(lambda x: (                        # categorize based on age
    'Children' if x[0] <= 17 else
    'Young Adults' if 18 <= x[0] <= 24 else
    'Adults' if 25 <= x[0] <= 64 else
    'Elderly People',
    1
    )) \
    .reduceByKey(lambda x, y: x + y)        # count based on key,meaning based on age group/x[0]
print(aggregated_data.collect())
# Format the results
end_time = time.time()
results.append(Row(
            Data_Abstractions="RDD",
            Execution_Time=end_time - start_time
        ))

results_df = spark.createDataFrame(results)
# Save the DataFrame to S3 as JSON
results_df.write.mode("overwrite").json("s3://groups-bucket-dblab-905418150721/group45/q1/results.json")


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3275,application_1732639283265_3231,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 4
Executor Memory: 4743M
Executor Cores: 2
[('Adults', 121093), ('Children', 10830), ('Young Adults', 33605), ('Elderly People', 5985)]